# Configuring cuDNN on Colab for YOLOv4


In [ ]:
# CUDA: Let's check that Nvidia CUDA drivers are already pre-installed and which version is it.
!/usr/local/cuda/bin/nvcc --version
# We need to install the correct cuDNN according to this output

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!nvidia-smi

Tue Jan 18 20:32:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Ensure to configure right architecture

In [ ]:
# This cell ensures you have the correct architecture for your respective GPU
# If you command is not found, look through these GPUs, find the respective
# GPU and add them to the archTypes dictionary

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# Tesla K80 
# ARCH= -gencode arch=compute_37,code=sm_37

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro RTX 8000, Quadro RTX 6000, Quadro RTX 5000, Tesla T4, XNOR Tensor Cores
# ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# Jetson XAVIER
# ARCH= -gencode arch=compute_72,code=[sm_72,compute_72]

# GTX 1080, GTX 1070, GTX 1060, GTX 1050, GTX 1030, Titan Xp, Tesla P40, Tesla P4
# ARCH= -gencode arch=compute_61,code=sm_61

# GP100/Tesla P100 - DGX-1
# ARCH= -gencode arch=compute_60,code=sm_60

# For Jetson TX1, Tegra X1, DRIVE CX, DRIVE PX - uncomment:
# ARCH= -gencode arch=compute_53,code=[sm_53,compute_53]

# For Jetson Tx2 or Drive-PX2 uncomment:
# ARCH= -gencode arch=compute_62,code=[sm_62,compute_62]
import os
os.environ['GPU_TYPE'] = str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read())

def getGPUArch(argument):
  try:
    argument = argument.strip()
    # All Colab GPUs
    archTypes = {
        "Tesla V100-SXM2-16GB": "-gencode arch=compute_70,code=[sm_70,compute_70]",
        "Tesla K80": "-gencode arch=compute_37,code=sm_37",
        "Tesla T4": "-gencode arch=compute_75,code=[sm_75,compute_75]",
        "Tesla P40": "-gencode arch=compute_61,code=sm_61",
        "Tesla P4": "-gencode arch=compute_61,code=sm_61",
        "Tesla P100-PCIE-16GB": "-gencode arch=compute_60,code=sm_60"

      }
    return archTypes[argument]
  except KeyError:
    return "GPU must be added to GPU Commands"
os.environ['ARCH_VALUE'] = getGPUArch(os.environ['GPU_TYPE'])

print("GPU Type: " + os.environ['GPU_TYPE'])
print("ARCH Value: " + os.environ['ARCH_VALUE'])

GPU Type: Tesla T4

ARCH Value: -gencode arch=compute_75,code=[sm_75,compute_75]


#Download darknet package and modify Makefile

In [ ]:
%cd /content/
%rm -rf darknet
!git clone https://github.com/AlexeyAB/darknet.git
%cd 


/content
Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 13.99 MiB | 25.18 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.
/root


In [ ]:
##Change your Makefile (from line 20-28) to the following lines
# USE_CPP=0
# DEBUG=0

# ARCH= -gencode arch=compute_35,code=sm_35 \
#       -gencode arch=compute_50,code=[sm_50,compute_50] \
#       -gencode arch=compute_52,code=[sm_52,compute_52] \
# 	    -gencode arch=compute_61,code=[sm_61,compute_61] \
#       -gencode arch=compute_37,code=sm_37

# ARCH= -gencode arch=compute_60,code=sm_60

# OS := $(shell uname)


In [ ]:
%cd /content/darknet/
!sed -i "1s/GPU=0/GPU=1/"               Makefile
!sed -i "2s/CUDNN=0/CUDNN=1/"           Makefile
!sed -i "4s/OPENCV=0/OPENCV=1/"         Makefile
!sed -i "5s/AVX=0/AVX=1/"               Makefile
!sed -i "7s/LIBSO=0/LIBSO=1/"           Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile
# !sed -i "21s/^/#/"                      Makefile
# !sed -i "22s/^/#/"                      Makefile
# !sed -i "23s/^/#/"                      Makefile
# !sed -i "24s/^/#/"                      Makefile
!sed -i "300s/calc_map_for_each = 4/calc_map_for_each = 1/" src/detector.c


/content/darknet


In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -ffp-contract=fast -mavx -mavx2 -msse3 -msse4.1 -msse4.2 -msse4a -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indenta

#Preparing step
###This step, you have to do it manually, (If you have some tricky ways please let me know)
####1. Prepare weight file (save in build\darknet\x64)
####2. Prepare dataset including: images and annotations (save in build\darknet\x64\data\obj\ #(create a obj folder))
####3. Prepare files including: 'modelname'.data, 'modelname'.names,train.txt, test.txt(save in build\darknet\x64\data\)
Your train.txt should contain path to images from checkpoint build/...
####4. Prepare configure files including: 'modelname'.cfg (save in build\darknet\x64\cfg); 

In [ ]:
# Name of the directory on Google Drive for saving training log, loss/mAP chart,
# and weights
SAVE_DIR = 'yolov4-tiny'

# Path of the Google Drive mount point
DRIVE_DIR = '/content/gdrive/MyDrive'

# Full path to the save directory on Google Drive
DRIVE_SAVE_DIR = DRIVE_DIR + '/' + SAVE_DIR



In [ ]:
#put your dataset in a compressed file.rar and push it on google drive
#link to your google drive and copy it to colab
from google.colab import drive
drive.mount('/content/gdrive')

from pathlib import Path
assert Path(DRIVE_DIR).is_dir(), 'Google Drive has not been mounted?'

%cd /content/


!mkdir -p "{DRIVE_SAVE_DIR}"
!rm -f "{DRIVE_SAVE_DIR}"/train.log
!touch "{DRIVE_SAVE_DIR}"/chart.png
!rm -f chart.png
!ln -sf "{DRIVE_SAVE_DIR}"/chart.png /content/darknet/chart.png
!mkdir -p "{DRIVE_SAVE_DIR}"/backup
!rm -rf backup
!ln -sf "{DRIVE_SAVE_DIR}"/backup /content/darknet/build/darknet/x64/



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [ ]:
!cp /content/gdrive/MyDrive/obj.zip /content/darknet/build/darknet/x64/data/
#Then decomspress it
%cd /content/darknet/build/darknet/x64/data/
!unzip obj.zip 


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: obj/image211.jpg        
  inflating: obj/image211.txt        
  inflating: obj/image212.jpg        
  inflating: obj/image212.txt        
  inflating: obj/image213.jpg        
  inflating: obj/image213.txt        
  inflating: obj/image214.jpg        
  inflating: obj/image214.txt        
  inflating: obj/image215.jpg        
  inflating: obj/image215.txt        
  inflating: obj/image216.jpg        
  inflating: obj/image216.txt        
  inflating: obj/image217.jpg        
  inflating: obj/image217.txt        
  inflating: obj/image218.jpg        
  inflating: obj/image218.txt        
  inflating: obj/image219.jpg        
  inflating: obj/image219.txt        
  inflating: obj/image22.jpg         
  inflating: obj/image22.txt         
  inflating: obj/image220.jpg        
  inflating: obj/image220.txt        
  inflating: obj/image221.jpg        
  inflating: obj/image221.txt        
  inflating: obj/image222.jpg  

In [ ]:
%cd obj/
%ls

In [ ]:
#To prepare train.txt, test.txt, there is a very useful way to do it:
#Preparing training and testing files
import glob2
import numpy as np
import os
all_files = []
# Move to your dataset directory
# Here is one of mine:
# %cd ImageAugmentation/
%cd /content/darknet/build/darknet/x64/data/obj/

for ext in ["*.png", "*.jpeg", "*.jpg"]:
#Remember to config your directory right here:
  images = glob2.glob(os.path.join("/content/darknet/build/darknet/x64/data/obj/", ext))
  all_files += images
print(all_files)
rand_idx = np.random.randint(0, len(all_files), int(len(all_files)*20/100))

# Create train.txt
with open("train.txt", "w") as f:
  for idx in np.arange(len(all_files)):
    if idx not in rand_idx:
      f.write(all_files[idx]+'\n')

# Create valid.txt
with open("test.txt", "w") as f:
  for idx in np.arange(len(all_files)):
    if idx in rand_idx:
      f.write(all_files[idx]+'\n')

/content/darknet/build/darknet/x64/data/obj
['/content/darknet/build/darknet/x64/data/obj/image344.jpg', '/content/darknet/build/darknet/x64/data/obj/image276.jpg', '/content/darknet/build/darknet/x64/data/obj/image333.jpg', '/content/darknet/build/darknet/x64/data/obj/image222.jpg', '/content/darknet/build/darknet/x64/data/obj/image475.jpg', '/content/darknet/build/darknet/x64/data/obj/frame1723.jpg', '/content/darknet/build/darknet/x64/data/obj/image301.jpg', '/content/darknet/build/darknet/x64/data/obj/image437.jpg', '/content/darknet/build/darknet/x64/data/obj/image239.jpg', '/content/darknet/build/darknet/x64/data/obj/frame1519.jpg', '/content/darknet/build/darknet/x64/data/obj/image466.jpg', '/content/darknet/build/darknet/x64/data/obj/frame1371.jpg', '/content/darknet/build/darknet/x64/data/obj/image389.jpg', '/content/darknet/build/darknet/x64/data/obj/frame160.jpg', '/content/darknet/build/darknet/x64/data/obj/image519.jpg', '/content/darknet/build/darknet/x64/data/obj/image30

In [ ]:
!mv "/content/darknet/build/darknet/x64/data/obj/train.txt" "/content/darknet/build/darknet/x64/data/"
!mv "/content/darknet/build/darknet/x64/data/obj/test.txt" "/content/darknet/build/darknet/x64/data/"


#Start training

In [ ]:
%cd /content/darknet
%ls
!./darknet detector train build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/obj_7000.weights -gpus 0 -map -dont_show 2>&1 | grep -E "hours left|mean_average"




 Last accuracy mAP@0.50 = 99.60 %, best = 99.61 % ]2;14145/15000: loss=0.2 map=1.00 best=1.00 hours left=0.2
 14145: 0.150630, 0.175250 avg loss, 0.000026 rate, 0.863257 seconds, 905280 images, 0.248174 hours left
 Last accuracy mAP@0.50 = 99.60 %, best = 99.61 % ]2;14146/15000: loss=0.2 map=1.00 best=1.00 hours left=0.2
 14146: 0.169377, 0.174663 avg loss, 0.000026 rate, 0.809967 seconds, 905344 images, 0.247743 hours left
 Last accuracy mAP@0.50 = 99.60 %, best = 99.61 % ]2;14147/15000: loss=0.2 map=1.00 best=1.00 hours left=0.2
 14147: 0.203402, 0.177537 avg loss, 0.000026 rate, 0.846670 seconds, 905408 images, 0.247353 hours left
 Last accuracy mAP@0.50 = 99.60 %, best = 99.61 % ]2;14148/15000: loss=0.3 map=1.00 best=1.00 hours left=0.2
 14148: 0.269205, 0.186704 avg loss, 0.000026 rate, 0.861627 seconds, 905472 images, 0.247094 hours left
 Last accuracy mAP@0.50 = 99.60 %, best = 99.61 % ]2;14149/15000: loss=0.2 map=1.00 best=1.00 hours left=0.2
 14149: 0.209141, 0.18894

#Evaluating


In [ ]:
%cd /content/darknet
# print('-------------------obj_last.weight-------------------------------')
# !./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/backup/obj_last.weights
# print('-------------------obj_best.weight-------------------------------')
# !./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/backup/obj_best.weights
# print('-------------------obj_4000.weight-------------------------------')
# !./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/backup/obj_4000.weights
# print('-------------------obj_5000.weight-------------------------------')
# !./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/backup/obj_5000.weights
# print('-------------------obj_6000.weight-------------------------------')
# !./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/backup/obj_6000.weights
# print('-------------------obj_7000.weight-------------------------------')
# !./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/backup/obj_7000.weights
print('-------------------obj_8000.weight-------------------------------')
!./darknet detector map build/darknet/x64/data/obj.data build/darknet/x64/cfg/obj.cfg build/darknet/x64/backup/obj_10000.weights


/content/darknet
-------------------obj_8000.weight-------------------------------
 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x

In [ ]:
##Wait signals
import time
i = 0
while (True):
  i = i+1
  time.sleep(10)
  print(i)

58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
